In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import analyze

In [ ]:
sim_dir = Path.cwd() / "src" / "sim"

freq = 2.45e9  # frequency in Hz
xns = np.array([1, 2, 4])  # number of antennas in x direction
yn = 1  # number of antennas in y direction
dxs = np.array([60, 90])  # distance between antennas in mm

fig = plt.figure(figsize=4 * np.array([dxs.size, xns.size]))
fig, axs = plt.subplots(
    nrows=xns.size,
    ncols=dxs.size,
    figsize=4 * np.array([dxs.size, xns.size]),
    subplot_kw={"projection": "polar"},
)
axs = axs.flatten()

for i, xn in enumerate(xns):
    for j, dx in enumerate(dxs):
        filename = f"farfield_{xn}x{yn}_{dx}x{dx}_{freq / 1e6:n}.h5"
        nf2ff = analyze.read_nf2ff(sim_dir / "antenna_array" / filename)
        E_norm, Dmax, theta = nf2ff["E_norm"][0][0], nf2ff["Dmax"], nf2ff["theta"]
        ax = axs[i * dxs.size + j]
        title = f"{xn}x{yn} array, {dx}x{dx}mm, {freq / 1e9:n}GHz"
        analyze.plot_ff_polar(E_norm, Dmax, theta, title=title, ax=ax)
fig.set_tight_layout(True)
fig.suptitle("OpenEMS Simulation Far-Field Patterns")
fig.savefig("antenna_array_ff.png", dpi=600)

In [ ]:
sim_dir = Path.cwd() / "src" / "sim"

filename = f"farfield_1x1_60x60_{freq / 1e6:n}.h5"
nf2ff = analyze.read_nf2ff(sim_dir / "antenna_array" / filename)
E_norm = nf2ff["E_norm"][0][0]
Dmax = nf2ff["Dmax"]
theta, phi = nf2ff["theta"], nf2ff["phi"]

freq = 2.45e9
yn = 1
xns = np.array([1, 2, 4])
dxs = np.array([60, 90])

fig, axs = plt.subplots(
    nrows=xns.size,
    ncols=dxs.size,
    figsize=4 * np.array([dxs.size, xns.size]),
    subplot_kw={"projection": "polar"},
)
axs = axs.flatten()
for i, xn in enumerate(xns):
    for j, dx in enumerate(dxs):
        dy = dx
        AF = analyze.array_factor(theta, phi[0], xn, yn, dx, dy, freq)
        E_norm_array = E_norm * AF.T
        title = f"{xn}x{yn} array, {dx}x{dy}mm spacing, {freq / 1e9:n}GHz"
        filename = f"array_{xn}x{yn}_{dx}x{dy}mm_{freq / 1e6:n}MHz"
        ax = axs[i * dxs.size + j]
        analyze.plot_ff_polar(E_norm_array, Dmax, theta, title=title, ax=ax)
fig.set_tight_layout(True)
fig.suptitle("Single Antenna & Array Factor Far-Field Patterns")
fig.savefig("array_factor.png", dpi=600)